In [1]:
import numpy as np
import pandas as pd
import scipy.stats as sps
import cvxpy
import matplotlib.pyplot as plt

from scipy.stats import norm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

from statsmodels.stats.outliers_influence import variance_inflation_factor

Напишем функции для подсчета коэффициента корреляции Пирсона и генерации матриц Sim и Rel.

In [2]:
def get_sim(X):
    return np.abs(np.corrcoef(X.T))

def get_rel(X, y):
    return np.abs(np.corrcoef(X.T, y)[:-1, -1])

Напишем функции для генерации данных.

In [3]:
def generate_data(size, eps, loc=0, scale=1):
    # генерируем y и x1 из стандартного нормального распределения
    
    # y - целевая переменная
    y = sps.norm(loc=loc, scale=scale).rvs(size=size)
    
    # x1 - первый признак
    x1 = sps.norm(loc=loc, scale=scale).rvs(size=size)

    # x2 - второй признак
    x2 = x1 + eps * y
    X = np.array([x1, x2]).T
    return X, y

In [4]:
def generate_noise_features(sample_size, n_features, loc=0, scale=1):
    X_noise = sps.norm(loc=loc, scale=scale).rvs(size=(sample_size, n_features))
    return X_noise

Напишем функцию для решения задачи квадратичного программирования.

In [5]:
def solve_qpfs(b, Q):
    a = cvxpy.Variable(Q.shape[1])

    problem = cvxpy.Problem(cvxpy.Maximize(b.T @ a - cvxpy.quad_form(a, Q)), [a >= 0])
    problem.solve(solver='ECOS')

    return a.value

Cгенерируем все данные: два главных и какое-то количество шумовых признаков.

In [6]:
def get_all_data(sample_size, noise_features, eps):
    X_main, y = generate_data(sample_size, eps)
    
    if noise_features > 0:
        X_noise = generate_noise_features(sample_size, noise_features)
        # соберем итоговую матрицу признаков, добавив шумовые признаки
        X = np.hstack((X_main, X_noise))
    else:
        # соберем итоговую матрицу признаков
        X = X_main
        
    return X, y

Напишем функцию для подсчета индекса мультиколлинеарности.

In [7]:
def get_VIF(X):
    # возвращаем максимальный из VIF по признакам в датасете
    return np.array([variance_inflation_factor(X, i) for i in range(X.shape[1])]).max()

Решим задачу оптимизации методом QPFS для отбора признаков.

\begin{cases}
z^* = \arg \min_{z \in [0, 1]^n}  z^T Q z - b^T z \\
\|z\|_1 \le 1
\end{cases}

In [8]:
def get_qpfs_experiment(X, y):
    
    # получим матрицы Rel и Q
    b = get_rel(X, y)
    Q = get_sim(X)
    
    # QBFS-метод
    a = solve_qpfs(b, Q)
    
    return a

Напишем функции для эксперимента с Ridge-регрессией и Lasso-регрессией.

In [9]:
def get_ridge(X_train, y_train, X_test, y_test, alpha):
    scaler = StandardScaler()

    # обучим этот класс на обучающей выборке
    scaler.fit(X_train)

    # применим стандартизацию к обучающей и тестовой выборкам
    X_train_stand = scaler.transform(X_train)
    X_test_stand = scaler.transform(X_test)
    
    model = Ridge(alpha=alpha)

    model.fit(X_train_stand, y_train)
    
    ridge_mse = mean_squared_error(model.predict(X_test_stand), y_test)
    
    # количество шумовых признаков в случае ridge-регрессии считаем как
    # количество признаков,коэффициенты при которых не очень близки к нулю
    curr_noise = (model.coef_ >= 0.000001).sum()
    
    ridge_VIF = get_VIF(X_train)
    
    return ridge_VIF, curr_noise, ridge_mse
    

In [10]:
def get_lasso(X_train, y_train, X_test, y_test, alpha):
    
    # стандартизируем признаки
    scaler = StandardScaler()

    # обучим этот класс на обучающей выборке
    scaler.fit(X_train)

    # применим стандартизацию к обучающей и тестовой выборкам
    X_train_stand = scaler.transform(X_train)
    X_test_stand = scaler.transform(X_test)
    
    model = Lasso(alpha=alpha)

    model.fit(X_train_stand, y_train)
    
    lasso_mse = mean_squared_error(model.predict(X_test_stand), y_test)
    
    lasso_VIF = get_VIF(X_train[:, model.coef_ != 0])
    
    curr_noise = (model.coef_[2:] != 0).sum()
    
    return lasso_VIF, curr_noise, lasso_mse

Также сделаем функции для qpfs-метода с разными порогами.

In [11]:
def get_qpfs_2_features(X_train, y_train, X_test, y_test):
    
    # qpfs (отбираем оба главных признака)
    best_a = get_qpfs_experiment(X_train, y_train)
    
    # устанавливаем порог tau как наименьшее значение среди best_a[0] и best_a[1], 
    # чтобы отобрать два признака и смотрим сколько шумовых признаков еще отобрали
    threshold = best_a[:2].min()
    
    # оставляем только отобранные признаки
    qpfs_X_train = X_train[:, best_a >= threshold]
    
    # cмотрим, сколько шумовых признаков отобрано
    curr_noise = qpfs_X_train.shape[1] - 2  # вычитаем 2, т.к. два основных признака точно отобрали
    
    # делаем обычную линейную регрессию
    model = LinearRegression()
    model.fit(qpfs_X_train, y_train)
    
    
    qpfs_mse = mean_squared_error(model.predict(X_test[:, best_a >= threshold]), y_test)
    
    qpfs_VIF = get_VIF(qpfs_X_train)
    
    return qpfs_VIF, curr_noise, qpfs_mse

In [12]:
def get_qpfs_some_features(X_train, y_train, X_test, y_test, threshold=0.00001):
    
    # qpfs (отбираем один главный признак)
    best_a = get_qpfs_experiment(X_train, y_train)
    
    # оставляем только отобранные признаки (cравниваем с переданным порогом threshold)
    qpfs_X_train = X_train[:, best_a >= threshold]
    
    # cмотрим, сколько шумовых признаков отобрано
    curr_noise = qpfs_X_train.shape[1] - (best_a[:2] >= threshold).sum()
    # берем только один основной признак
    
    # делаем обычную линейную регрессию
    model = LinearRegression()
    model.fit(qpfs_X_train, y_train)
    
    qpfs_mse = mean_squared_error(model.predict(X_test[:, best_a >= threshold]), y_test)
    
    qpfs_VIF = get_VIF(qpfs_X_train)
    
    return qpfs_VIF, curr_noise, qpfs_mse

Напишем общую функцию для запуска эксперимента.

In [13]:
def get_experiment(train_sample_size, noise_features, eps, test_size, alpha=0.01, threshold=0.00001):
    # сгенерируем данные для обучения
    X_train, y_train = get_all_data(train_sample_size, noise_features, eps)
    
    # сгенерируем данные для теста
    X_test, y_test = get_all_data(test_size, noise_features, eps)
    
    qpfs_2_features = np.array(get_qpfs_2_features(X_train, y_train, 
                                                   X_test, y_test))
    
    qpfs_some_features = np.array(get_qpfs_some_features(X_train, y_train, 
                                                         X_test, y_test, threshold))
    
    ridge_res = np.array(get_ridge(X_train, y_train, X_test, y_test, alpha))
    
    lasso_res = np.array(get_lasso(X_train, y_train, X_test, y_test, alpha))
    
    res = np.vstack((qpfs_2_features, qpfs_some_features, ridge_res, lasso_res))
    
    
    return res

In [14]:
def show_results(res_matrix):
    return pd.DataFrame(res_matrix,
                        index=['QPFS (с отбором двух главных признаков)', 'QPFS c фикс. порогом', 
                               'Ridge', 'Lasso'], 
                        columns=['VIF', 'Количество шумовых признаков', 
                                 'MSE на тестовой выборке'])

### Данные для эксперимента

In [15]:
train_sample_size = 1000
noise_features = 10
eps = 0.001

# коэф-т регуляризации для Ridge и Lasso
alpha = 0.01

test_size = 100

In [16]:
# проведем один эксперимент
show_results(get_experiment(train_sample_size, noise_features, eps, test_size))

,VIF,Количество шумовых признаков,MSE на тестовой выборке
QPFS (с отбором двух главных признаков),1.068213e+06,10.0,1.050598e-24
QPFS c фикс. порогом,1.016200e+00,10.0,1.039584e+00
Ridge,1.068213e+06,5.0,9.485545e-01
Lasso,1.016196e+00,9.0,1.031756e+00


А теперь сделаем несколько экспериментов и усредним результаты.

In [17]:
num_attempts = 100

In [18]:
# усредним результаты
mean_results = np.array([get_experiment(train_sample_size, 
                                        noise_features, eps, test_size) 
                         for i in range(num_attempts)]).mean(axis=0)

In [19]:
# выведем усредненные значения
show_results(mean_results)

,VIF,Количество шумовых признаков,MSE на тестовой выборке
QPFS (с отбором двух главных признаков),1.023882e+06,8.96,1.170779e-23
QPFS c фикс. порогом,1.014672e+00,8.74,1.045660e+00
Ridge,1.023911e+06,5.90,9.505797e-01
Lasso,1.012684e+00,7.55,1.040215e+00
